# Radiomics for experts
  
<b>'Building a radiomics classifier that distinguishes between benign and malignant lesions will certainly improve clinical handling of suspicious lesions, and minimize the need for invasive procedures' - the aim of all the radiomics researchers.</b>

## Introduction

In this script, you will use a privately collected contrast enhanced spectral mammography (CESM) of 1058 patients at Maastricht University Medical Center (MUMC). You will build a classifier to differentiate between malignant and benign breast lesions. Such a model will reduce the necessity for invasive tissue biopsies. If you find this task, or parts of it, too hard, you can "cheat" by looking at the beginners notebook, as most tasks are very similar.

<b>Note:</b> this dataset is private and allowed to be used only for the purpose of this course! Nevertheless, for this notebook, we provide you with the features extracted from the data as an invert transformation is not possible. If you want to extract the features yourself, you will receive the dedicated instructions at the practical. In this case, please make sure you delete imaging data after the workshop. For this, you can use a precision-medicine-toolbox: https://github.com/primakov/precision-medicine-toolbox. For the hints, you can have a look at the beginners notebook: https://colab.research.google.com/drive/1K-sg8g2Po6Czq5utFS2F6CtetXcrwpIt?usp=sharing.

<b>Brief description</b>

Mammography is the standard imaging tool for breast-cancer screening, but is prone to false positives and lowered sensitivity in high-density breasts. Contrast enhanced spectral mammography (CESM) depicts breast lesions with more accuracy and with higher sensitivity than regular mammograms. This technique uses an iodine contrast agent in addition to a high energy and low energy mammogram to cancel out background noise and enhance the tumor. The advantage of CESM compared to regular mammography is that breast density does not affect image quality, and it boasts a sensitivity of 93-100%, compared to 75-85% of mammography on regular breasts, and lower than 50% in dense breasts.   

![picture](https://drive.google.com/uc?export=view&id=1GRt1gWtxPDXGycX6TpDAbH1XAieMQc0t)      
A typical contrast-enhanced spectral mammography (CESM) examination (only right mediolateral oblique view shown), consisting of a low-energy (a), high-energy (b) and recombined (c) image. A suspicious lesion is seen on the low-energy image, showing enhancement on the recombined image (white arrows). Histopathology showed invasive ductal carcinoma. The high-energy image is not for diagnostic purposes but is used for construction of the recombined image (Lalji, U.C., Jeukens, C.R.L.P.N., Houben, I. et al. Evaluation of low-energy contrast-enhanced spectral mammography images by comparing them to full-field digital mammography using EUREF image quality criteria. Eur Radiol 25, 2813–2820 (2015). https://doi.org/10.1007/s00330-015-3695-2)

<b>Problem</b>  

Though CESM improves the detection of breast lesions, there is still a challenge in distinguishing between benign and malignant lesions, as the specificity of CESM is estimated to be 70%. This stands to improve using machine learning (ML) algorithms.

<b>Notebook structure</b>  

The Python script will take you through the following steps:  
 - Installing packages and importing libraries,
 - Reading the data and assigning the outcomes,
 - Removing redundant features and selecting predictive features,
 - Creating the classification models.
If you still have time, go back and try balancing your data, and fine-tune your models.

## Getting started

This is an interactive Python notebook. To run it, you don't need to install anything on your PC since the script is executed in the cloud. On the left tab, you can see the 'Files' folder, that contains the data. Results of the script execution will be saved in this folder.  
  
The information about the curated imaging dataset will be distributed separately. To open DICOM files, you can use 3D Slicer (https://www.slicer.org/), RadiAnt viewer (https://www.radiantviewer.com/), or MicroDicom viewer (https://www.microdicom.com/downloads.html). As feature extraction process takes time, we prepared the Excel tables with the pre-extracted features to work with. All the data will be pulled from the shared Google Drive to the temporary environment of this notebook. Further it is explained how to get these data.

Please note, that all the files you pull or upload to this notebook, as well as the files, produced while executing the script, are automatically deleted as soon as you end the session.

First of all, the needed Python packages have to be uploaded. Some of them are not installed in the environment of this notebook, so the installation is needed with '!pip install <i>name-of-the-package</i>' command. We recommend you get acquainted with getting documentation and help on these packages. For example, google 'python sklearn' and you will get to the documentation quickly. Importing libraries is a necessary step with most progamming languages, not only Python.

In [ ]:
# installing some packages, which are not part of the present Google Collab environment

!pip install precision-medicine-toolbox

In [ ]:
import os
import numpy as np
import pandas as pd
from pmtool.AnalysisBox import AnalysisBox
from pmtool.ToolBox import ToolBox
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
import seaborn as sns
import xgboost as xgb



Pulling the shared files and unzipping (pay attention at 'Files' tab of this notebook):

In [ ]:
!gdown --id 1hrVNUSWmqEgLcX2slkENoCzPA9aqKy16
!unzip AI4I_Radiomics_Experts_Data.zip

## Feature extraction
There are many softwares to perform this step, each with their strengths and weaknesses. Features extraction step takes the longest time, so we are happy to help you with it. We will supply you with "something we prepared earlier", the lists of features already extracted with Radiomics software, and outcomes (benign/malignant). The data is already split into train and validation sets. Therefore, you can skip this step or return to it later. If you want to skip this step, please move to 'Reading the data' step.  

If you want to extract the features by yourself anyway, you can adapt the code from the beginners notebook: https://colab.research.google.com/drive/1K-sg8g2Po6Czq5utFS2F6CtetXcrwpIt?usp=sharing. Here we perform features extraction with PyRadiomics software (https://pyradiomics.readthedocs.io/en/latest/) with the precision-medicine-toolbox (https://github.com/primakov/precision-medicine-toolbox) interface. These are the open-source Python packages. The key difference is that the present script is using Pyradiomics for feature extraction whereas in the prepared files, Radiomics software (https://radiomics.bio/) was used.

## Reading the data

For this step, you will need to specify the path to the files containing the features and outcomes (in this case 0 or 1 for benign or malignant). It is important that apart from initial dataset cleaning, all the work is only performed in the training dataset and the test set remains untouched.

<b>NOTE: the separator character, text quotation, decimal point character change depending on your country settings. If this step fails, add those settings into read_excel function. Also, the outputs from different radiomics softwares can differ wildly, so some thought is necessary when doing this at home.</b>  



In [ ]:
data_train = pd.read_excel("/content/AI4I_Radiomics_Experts_Data/data_features/WS2022_Experts_training.xlsx")
data_test = pd.read_excel("/content/AI4I_Radiomics_Experts_Data/data_features/WS2022_Experts_validation.xlsx")

# let's have a look at our dataframe:
data_train


<b>Getting the features and defining the outcome</b>

It's a good time to look at the .csv files using Google Tables or similar software. Ask a helper if you need assistance with this. Let's have a look at the columns and get a lis of the features from there:

In [ ]:
counter = 0
for item in data_train.columns:
  print (counter, item)
  counter += 1

And get list of the features: print features names from the provided file above


In [ ]:
features = list(............)
print (len(features))
print (features)

Separate the features and the outcomes:

In [ ]:
outcome_train = data_train['.........']
outcome_test = data_test['..........']
data_train = data_train[........]
data_test = data_test[.........]

# checking how the features dataframe looks like
data_test

## Exploratory data analysis

Before building the models, it is always useful to perform the exploratory data analysis to have a look at the data, understand the distribution of the features, and notice some data errors. To perform these steps, we will be using precision-medicine-toolbox (https://github.com/primakov/precision-medicine-toolbox), an open-source Python package, developed in the D-Lab, Maastricht University. If something does not work, look at the error and at the already produced features reports for troubleshooting. Don't hesitate to look into the toolbox documentation to fix the problem: https://precision-medicine-toolbox.readthedocs.io/en/latest/?badge=latest. You can search for available functions to deal with NaN values + the ways to read the data, excluding some features.  
  
Remember that for now, everything happens in the training data. Time to discuss why this is so important!  

In [ ]:
# let's list the parameters of our features dataset
parameters = {
    'feature_path': "/content/AI4I_Radiomics_Experts_Data/data_features/WS2022_Experts_training.xlsx", # path to csv/xls file with features
    'outcome_path': "/content/AI4I_Radiomics_Experts_Data/data_features/WS2022_Experts_training.xlsx", #path to csv/xls file with outcome (in our case - the same file)
    'patient_column': 'LesionID', # name of column with patient ID
    'patient_in_outcome_column': 'LesionID', # name of column with patient ID in clinical data file
    'outcome_column': 'Outcome', # name of outcome column
    'feature_column': features,
    'feature_column_to_drop': ['LoG_sigma_0_4_mm_2D_IH_max',
                               'LoG_sigma_0_7_mm_2D_IH_max',
                               'LoG_sigma_0_4_mm_2D_pos_IH_range',
                               'LoG_sigma_0_4_mm_2D_pos_IH_min',
                               'LoG_sigma_0_1_mm_2D_pos_IH_max',
                               'LoG_sigma_1_mm_2D_IH_max',
                               'LoG_sigma_0_7_mm_2D_IH_range',
                               'LoG_sigma_1_mm_2D_pos_IH_max',
                               'LoG_sigma_0_4_mm_2D_IH_min',
                               'LoG_sigma_0_1_mm_2D_pos_IH_range',
                               'LoG_sigma_0_1_mm_2D_IH_range',
                               'LoG_sigma_1_mm_2D_IH_min',
                               'LoG_sigma_0_4_mm_2D_IH_range',
                               'LoG_sigma_1_mm_2D_pos_IH_range',
                               'voxel_distance',
                               'LoG_sigma_0_1_mm_2D_IH_max',
                               'LoG_sigma_0_7_mm_2D_pos_IH_max',
                               'LoG_sigma_1_mm_2D_IH_range',
                               'LoG_sigma_0_4_mm_2D_pos_IH_max',
                               'LoG_sigma_0_7_mm_2D_pos_IH_min',
                               'LoG_sigma_1_mm_2D_pos_IH_min',
                               'LoG_sigma_0_1_mm_2D_pos_IH_min',
                               'LoG_sigma_0_7_mm_2D_IH_min',
                               'LoG_sigma_0_7_mm_2D_pos_IH_range',
                               'LoG_sigma_0_1_mm_2D_IH_min']
}

# initializing the dataset, containing features
fs = AnalysisBox(**parameters)

In [ ]:
print ('Initial amount of features: ', len(..........))
fs.handle_nan(axis=1)
print ('Amount of features after exclusion of NaN values: ', len(.............))

Plotting feature distributions to have a visual understanding about feature distributions in classes.

In [ ]:
fs............................

What do you think about the report?

Plotting the correlation matrix to have an idea about how many features are inter-correlated. Why is it important to know about the mutual feature correlation?

In [ ]:
fs..............................

Performing Mann-Whitney U-test (with False Discovery Rate correction) to see if the feature distributions in classes are statistically different.

In [ ]:
fs................................

Plotting univariate ROC curves and calculating AUC scores to have some estimation of the predictive power of every separate feature. Can we just build a classifer based on the best feature?

In [ ]:
auc_threshold=0.70
fs.plot_univariate_roc(................ , ..............)

Writing the basic statistics for every feature into .csv file and having a look at the resulting dataframe.

In [ ]:
fs.calculate_basic_stats()

print('Basic statistics for each feature')
pd.read_excel('/content/AI4I_Radiomics_Experts_Data/data_features/WS2022_Experts_training_basic_stats.xlsx')

## Features reduction and selection

Do you think we need to change somehow features list, based on the exploratory analysis?

In [ ]:
features_non_nan = data_train[features].dropna(axis=1).columns
print ('Number of features without missing values: ', len(...........))

features_non_zero_var = data_train[features_non_nan].loc[:,data_train[features_non_nan].std() > 0.3].columns
print ('Number of features with non-zero variance: ', len(...............))

Removing highly correlated features is a controversial step aimed to reduce the dimensionality of the feature space. Highly correlated features needlessly inflate the dimensionality of feature space. The idea is that highly correlated features can be grouped together and represented by one representative feature.  For features pairs with a high Spearman correlation (r > 0.9) the feature with the highest mean correlation with all remaining features is removed.
The opponents to this step argue that just because features are correlated doesn't mean that they don't individually  increase the model's performance.
Time to discuss the pro's and con's of this step in more detail!

<b>If you have time at the end of today's workshop, the cutoff is certaily a variable that can be played with.</b>

In [ ]:
def selectNonIntercorrelated(df_in, ftrs, corr_th):

    # selection of the features, which are not 'highly intercorrelated' (correlation is defined by Spearman coefficient);
    # pairwise correlation between all the features is calculated,
    # from each pair of features, considered as intercorrelated,
    # feature with maximum sum of all the pairwise Spearman correlation coefficients is a 'candidate to be dropped'
    # for stability of the selected features, bootstrapping approach is used:
    # in each bootstrap split, the random subsample, stratified in relation to outcome,
    # is formed, based on original observations from input dataset;
    # in each bootstrap split, 'candidates to be dropped' are detected;
    # for each input feature, its frequency to appear as 'candidate to be dropped' is calculated,
    # features, appeared in 50 % of splits as 'candidate to be dropped', are excluded from feature set

    # input:
    # df_in - input dataframe, containing feature values (dataframe, columns = features, rows = observations),
    # ftrs - list of dataframe features, used in analysis (list of feature names - string variables),
    # corr_th - threshold for Spearman correlation coefficient, defining each pair of features as intercorrelated (float)

    # output:
    # non_intercorrelated_features - list of names of features, which did not appear as inter-correlated

    corr_matrix = df_in.corr(method='spearman').abs()
    mean_absolute_corr = corr_matrix.mean()
    intercorrelated_features_set = []
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    high_corrs = upper.where(upper > corr_th).dropna(how='all', axis=1).dropna(how='all', axis=0)

    for feature in high_corrs.columns:
        mean_absolute_main = mean_absolute_corr[feature]
        correlated_with_feature = high_corrs[feature].index[pd.notnull(high_corrs[feature])]
        for each_correlated_feature in correlated_with_feature:
            mean_absolute = mean_absolute_corr[each_correlated_feature]
            if mean_absolute_main > mean_absolute:
                if feature not in intercorrelated_features_set:
                    intercorrelated_features_set.append(feature)
            else:
                if each_correlated_feature not in intercorrelated_features_set:
                    intercorrelated_features_set.append(each_correlated_feature)

    non_intercorrelated_features_set = [e for e in ftrs if e not in intercorrelated_features_set]

    print ('Non intercorrelated features: ', non_intercorrelated_features_set)

    return non_intercorrelated_features_set

We will use a threshold of 0.9 for absolute value of Spearmann's correlation.

In [ ]:
features_non_intercorrelated = selectNonIntercorrelated(.............., ......................, 0.9)
print ('Number of non-intercorrelated features: ', len(.....................))

We will perform feature selection using Recursive Feature Elimination (RFE). In this step, feature selection is based on the outcome, so simple models are built and those features that contribute the least to the model are removed recursively. Here you can edit the parameters.

<b>You might have heard of variable normalization. Why are we not normalizing the variables (e.g. Z-score normalization)?</b>
Hint: e.g. https://stackoverflow.com/questions/57339104/is-normalization-necessary-for-randomforest

<b>How many features do we need to select with RFE?
Time to discuss pros and cons of using many and a little features.</b>  
  
There are some rules of thumb on how many features we need in the end:  
* $int(\frac{N_{samples}}{10})$ (Abu-Mostafa, Y. S., Magdon-Ismail, M., & Lin, H. T. (2012). Learning from data (Vol. 4, p. 4). New York: AMLBook.)  
* $\sqrt{N_{samples}}$ (Hua, J., Xiong, Z., Lowey, J., Suh, E., & Dougherty, E. R. (2005). Optimal number of features as a function of sample size for various classification rules. Bioinformatics, 21(8), 1509-1515.)

In [ ]:
print ('Nmber of samples in training dataset: ', ..........................)
print ('Number of features to select according to Abu-Mostafa: ',......................... )
print ('Number of features to select according to Hua: ', ..............................)

Let's go for the lower number of features since our dataset is not large. Below we implement Recursive Feature Elimination, RFE (https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html), based on Random Forest Classifier, RFC (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Which number of features will you chose?

In [ ]:
estimator = RandomForestClassifier(n_estimators=100, random_state=np.random.seed(0))
selector = RFE(estimator, n_features_to_select=20, step=1)
selector = selector.fit(data_train[features_non_intercorrelated], outcome_train)

And then select the features 'supported' by the algorithm.

In [ ]:
support = selector.get_support()
selected_features_set = data_train[features_non_intercorrelated].loc[:, support].columns.tolist()

print (selected_features_set)

<b>How many features give the best performance?</b>

Time to discuss why we don't just use 100 features!  
  
The other option to select a number of features is data-driven and is based on ranking the features and then building classifier adding +1 feature at every step and checking its performance. We can evaluate the performance with some score and plot the dependence of this score from the number of the selected features.

In [ ]:
# ranking the features based on RFE results

features_ranks = pd.DataFrame({'Features': features_non_intercorrelated, 'Ranks': selector.ranking_})
features_ranks.sort_values(by='Ranks', inplace = True)

# taking one best feature first, building the RFC, estimating the performance;
# adding +1 next feature, repeating the steps to estimate the performance

ftrs_number_tuning = []
acc_tuning = []

for i in range (1, len(features_ranks)):

    ftrs_number_tuning.append(i)
    estimator_tuning = RandomForestClassifier(n_estimators=100, random_state=np.random.seed(0))
    estimator_tuning.fit(data_train[features_ranks['Features'][:i]], outcome_train)
    outcome_pred_tuning = estimator_tuning.predict(data_test[features_ranks['Features'][:i]])
    acc_tuning.append(balanced_accuracy_score(outcome_test, outcome_pred_tuning))

# plotting the results
plt.plot(ftrs_number_tuning, acc_tuning)
plt.xlabel('Number of features')
plt.ylabel('RFC performance')
plt.title('RFC performance depending on number of selected features')
plt.show()

What can we conclude from the plot? What are the downsides of the presented implementation? Is it correct to train and evaluate the model on the same samples? Is the selected performance metric correct? What other metrics can we use?

## Modeling



### MODEL 1: RFC

We started with RFC while performing RFE, so let us train this model with the selected features and evaluate in on test data.  

Training the model (it's possible to vary the parameters):

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=np.random.seed(0))
rfc.fit(...............[selected_features_set], ..............)

Prediction for the testing set:

In [ ]:
outcome_pred_rfc = rfc.predict(.........................................)

Performance reporting on some key classification scores:

In [ ]:
print (classification_report(..............., .................))

Are precision and recall informative metrics? Why don't we report accuracy as a key metric? In which cases accuracy is not suitable scores?  
  
The other popular metric for classification is Receiver Operating Characteristic (ROC) curve and area under the curve (AUC). We will calculate true positive rates (TPR) and false positive rates (FPR) while varying classification threshold and plot the curve.

In [ ]:
fpr, tpr, _ = roc_curve(outcome_test, rfc.predict_proba(data_test[selected_features_set])[:, 1])
roc_auc = roc_auc_score(...........................................................)

plt.plot(fpr, tpr)
plt.title('RFC ROC curve (AUC = {})'.format(roc_auc))
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In which cases this metric does not give a correct representation of the model performance? Is AUC always the best metric?
Most cetainly not! Especially for unbalanced datasets, AUC can be meaningless.
http://www.davidsbatista.net/blog/2018/08/19/NLP_Metrics/
"With imbalanced classes, it’s easy to get a high accuracy without actually making useful predictions. So, accuracy as an evaluation metrics makes sense only if the class labels are uniformly distributed"    

To have a better understanding of model behaviour, we will plot a confusion matrix.

In [ ]:
cm = confusion_matrix(................., .....................)
f = sns.heatmap(cm, annot=True)
plt.title('Confusion matrix for RFC')
plt.show()



### MODEL 2: XGBoost

The second model we will build is XGBoost because recently the algorithm was successfull in many machine learning competitions.

<b>XGBoost - what is it? It's always best to understand your models. </b>

https://xgboost.readthedocs.io/en/stable/python/python_intro.html#

Train the model: Here you can chose the max number of boosting iterations, a balance between computing time and accuracy.

Presenting the data in the appropriate format for the library.

In [ ]:
dtrain = xgb.DMatrix(data_train[selected_features_set], label=............)
dtest = xgb.DMatrix(............[selected_features_set], label=............)

Defining the parameters: the learning objective is logistic regression for binary classification with probability output, the metric is the area under precision-recall curve (why not ROC AUC?).

In [ ]:
param = {'objective': 'binary:logistic', 'eval_metric': 'aucpr'}

We will train the model on the training set and evaluate on test set.

In [ ]:
evallist = [(dtest, 'eval'), (dtrain, 'train')]

Training the model (number of iterations can be changed here!) and calculating outcomes for the test set.

In [ ]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

outcome_pred_xgb = bst.predict(dtest)

Classification report:

In [ ]:
print (classification_report(............., .................>0.5))

ROC and ROC AUC:

In [ ]:
fpr, tpr, _ = roc_curve(.............., ..............)
roc_auc = roc_auc_score(outcome_test, ................)

plt.plot(fpr, tpr)
plt.title('XGBoost ROC curve (AUC = {})'.format(roc_auc))
plt.show()

Create and display the confusion matrix and derived values for the Xgboost model.

In [ ]:
cm = confusion_matrix(.............., .............>0.5)
f = sns.heatmap(cm, annot=True)

<b>Almost done! You will now compare the performance of the models.</b>
  
Which classifier is better?  

To compare ROC AUC scores, we will perform a permuation test for the probabilities obtained on the test set for the both classifiers.

In [ ]:
# adapted from https://stackoverflow.com/questions/52373318/how-to-compare-roc-auc-scores-of-different-binary-classifiers
#-and-assess-statist

def permutation_test_between_clfs(y_test, pred_proba_1, pred_proba_2, nsamples=100):
    auc_differences = []
    auc1 = roc_auc_score(y_test.ravel(), pred_proba_1.ravel())
    auc2 = roc_auc_score(y_test.ravel(), pred_proba_2.ravel())
    observed_difference = auc1 - auc2
    for _ in range(nsamples):
        mask = np.random.randint(2, size=len(pred_proba_1.ravel()))
        p1 = np.where(mask, pred_proba_1.ravel(), pred_proba_2.ravel())
        p2 = np.where(mask, pred_proba_2.ravel(), pred_proba_1.ravel())
        auc1 = roc_auc_score(y_test.ravel(), p1)
        auc2 = roc_auc_score(y_test.ravel(), p2)
        auc_differences.append(auc1 - auc2)
    return observed_difference, np.mean(auc_differences >= observed_difference)

print ('Difference, p-value: ',
       permutation_test_between_clfs(outcome_test,
                                     outcome_pred_xgb,
                                     rfc.predict_proba(data_test[selected_features_set])[:,1]))

After this test, which classifier is better?  
  
<b>Time for a pre-hackathon!</b> You can go back and "finetune" the models (or even implement your models) by changing the parameters you feel comfortable with. See you can increase the model performance. Can you beat the other groups?